## Battery Aging Challenge
Estimating the remaining capacity form field data.

### Input
* 6 months of battery voltage, battery current and battery temperature (stored in monthly .csv files: 2020-04 to 2020-10)
* regular testbench capacity measurements for the first 3 months
* chemistry: NMC

### Task
Estimate the remaining capacity at the following dates:
* 2020-07-11
* 2020-08-04
* 2020-08-28
* 2020-09-22
* 2020-10-01

### Gotcha
Capacity test related time series were removed from the data to prevent the solution of plain coulomb counting for those tests. Therefore you will recognize data gaps at certain dates.

In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from utils import plot_battery_data, plot_testbench_results
import numpy as np
%matplotlib widget  
from ipywidgets import *
import matplotlib.pyplot as plt


In [ ]:
battery_data = pd.concat([pd.read_csv('data/battery_data/2020-{}.csv'.format(month)) for month in range(4, 10)])

battery_data['datetime'] = pd.to_datetime(battery_data['timestamp'], unit='s')
# cutting out 2 days
subset = battery_data[battery_data['timestamp']<datetime.datetime(2020, 4, 29, 0, 0, 0).timestamp()]


In [ ]:
plt.figure()
plt.plot(subset['datetime'], subset['voltage'])
plt.show()

In [ ]:
motif = battery_data[(datetime.datetime(2020, 4, 2, 11, 34, 3).timestamp() < battery_data['timestamp']) &
                     (datetime.datetime(2020, 4, 3, 5, 1, 17).timestamp() > battery_data['timestamp'])]

plt.figure()
plt.plot(motif['datetime'], motif['voltage'])

In [ ]:
sampling = 100

data, motif = battery_data[::sampling]['voltage'].to_numpy(), motif[::sampling]['voltage'].to_numpy()
motif = motif - np.mean(motif)
key = 'voltage'

l = len(motif)
errors = []
for start in range(data.shape[0] - l):
    samp = data[start:start + l] - np.mean(data[start:start + l])
    errors.append(np.mean(abs(samp - motif)))

In [ ]:
plt.figure()
plt.plot(errors)
plt.show()

In [ ]:
error_cutoff = 0.02
dataset = []
plt.figure()
prev_start = -l
for start, error in enumerate(errors):
    if (error < error_cutoff) & (start - l >= prev_start):
        prev_start = start
        plt.plot(data[start:start + l])
        dataset.append(data[start:start + l])
plt.show()


In [ ]:
len(dataset)
plt.figure()
plt.imshow(np.array(dataset))